In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import requests
import xmltodict

In [2]:
authKey = 'G7GuVfz99A0umUDOAuZlRXMYlZmKxBd3hHsj8EiZ89SSD7r5qrn08q70LM96fyblxiM3eGxWsgkIjhF%2BAWTVBg%3D%3D'

## APT 매매이력 API
def get_apt_trading(sigungu_cd, base_month, authKey = authKey):
    url = f"http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey={authKey}&pageNo=1&numOfRows=100000&LAWD_CD={sigungu_cd}&DEAL_YMD={base_month}"
    try:
        response = requests.get(url)
        raw = xmltodict.parse(response.text)
        raw = raw['response']['body']['items']['item']
        return pd.DataFrame(raw)
    except Exception as e:
        return print(f'기준년월: {base_month}, 시군구코드: {sigungu_cd} 결과를 가져오는데 실패했습니다.')

In [3]:
## 법정동코드 불러오기
dl_path = 'C:/Users/GCC/git/data/DL/'
bjd_cd = pd.read_csv(f'{dl_path}법정동코드 전체자료.txt', encoding='cp949', sep='\t', dtype=str)
bjd_cd['시군구코드'] = bjd_cd['법정동코드'].str.slice(0, 5)
bjd_cd['시도코드'] = bjd_cd['법정동코드'].str.slice(0, 2)

In [4]:
## 조회조건 정의
sigungu_cd_ls = (bjd_cd[((bjd_cd['시도코드'] + '000') != bjd_cd['시군구코드'])&(bjd_cd['폐지여부'] == '존재')&
                        ((bjd_cd['시도코드'] == '11')|(bjd_cd['시도코드'] == '41'))]['시군구코드']).unique()
base_month_ls = pd.date_range('2006-01-01', '2024-01-31', freq='M').strftime('%Y%m').tolist()

In [5]:

import concurrent.futures
_ls = []
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    for sigungu_cd in sigungu_cd_ls:
        print(f'{sigungu_cd} 시작')
        future_month = {executor.submit(get_apt_trading, sigungu_cd, month): month for month in base_month_ls}
        for future in concurrent.futures.as_completed(future_month):
            dataset = future.result()
            _ls.append(dataset)
        print(f"done {sigungu_cd}")
trading_df = pd.concat(_ls)

11110 시작
done 11110
11140 시작
done 11140
11170 시작
done 11170
11200 시작
done 11200
11215 시작
done 11215
11230 시작
done 11230
11260 시작
done 11260
11290 시작
done 11290
11305 시작
done 11305
11320 시작
done 11320
11350 시작
done 11350
11380 시작
done 11380
11410 시작
done 11410
11440 시작
done 11440
11470 시작
done 11470
11500 시작
done 11500
11530 시작
done 11530
11545 시작
done 11545
11560 시작
done 11560
11590 시작
done 11590
11620 시작
done 11620
11650 시작
done 11650
11680 시작
done 11680
11710 시작
done 11710
11740 시작
done 11740
41110 시작
기준년월: 200601, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200702, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200602, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200611, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200605, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200609, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200606, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200705, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200703, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200603, 시군구코드: 41110 결과를 가져오는데 실패했습니다.
기준년월: 200612, 시군구코드: 41110 결과를 가져오는데 실패했습

In [7]:
dl_path

'C:/Users/GCC/git/data/DL/'

In [8]:
trading_df.to_csv(f'{dl_path}/매매내역_서울경기.csv', index=False, encoding='utf-8-sig')